<a href="https://colab.research.google.com/github/JASONZ777/senti_analy_prompt-BERT-LoRA/blob/main/senti_ana.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Method 1: Prompt-BERT (add a classification layer)

## Design a hard prompt

In [ ]:
# define a hard prompt, and its ID
def get_prompt(x):
    prompt = f'总体上来说很[MASK]: {x}'
    return {
        'prompt': prompt,
        'mask_id': prompt.find('[MASK]') # since only one [MASK], find the ID
    }

# map the label to the word in the dictionary
# (tokenizer, words with similiar meanings have similar embeddings, that's why we could calculate the loss)
def map_label(tokenizer):
    return {
        '1': {'token': '好', 'id': tokenizer.convert_tokens_to_ids("好")},
        '0': {'token': '差', 'id': tokenizer.convert_tokens_to_ids("差")}
    }


In [ ]:
# a testing from Huggingface
from transformers import AutoTokenizer

checkpoint = "bert-base-chinese"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

comment = '这个宾馆比较陈旧了，特价的房间也很一般。'

print('verbalizer:', map_label(tokenizer))

prompt_data = get_prompt(comment)
prompt, mask_offset = prompt_data['prompt'], prompt_data['mask_id']

encoding = tokenizer(prompt, truncation=True)
tokens = encoding.tokens()
mask_idx = encoding.char_to_token(mask_offset)

print('prompt:', prompt)
print('prompt tokens:', tokens)
print('mask idx:', mask_idx)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/624 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/110k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/269k [00:00<?, ?B/s]

verbalizer: {'1': {'token': '好', 'id': 1962}, '0': {'token': '差', 'id': 2345}}
prompt: 总体上来说很[MASK]: 这个宾馆比较陈旧了，特价的房间也很一般。
prompt tokens: ['[CLS]', '总', '体', '上', '来', '说', '很', '[MASK]', ':', '这', '个', '宾', '馆', '比', '较', '陈', '旧', '了', '，', '特', '价', '的', '房', '间', '也', '很', '一', '般', '。', '[SEP]']
mask idx: 7


## Upload dataset combined with prompts

In [ ]:
from torch.utils.data import Dataset

class upload_dataset(Dataset):
    def __init__(self, data_file):
        self.data = self.load_data(data_file)

    def load_data(self, data_file):
        Data = {}
        with open(data_file, 'rt', encoding='utf-8') as f:
            for idx, line in enumerate(f):
                items = line.strip().split('\t')
                prompts = get_prompt(items[0])
                assert len(items) == 2
                Data[idx] = {
                    'comment': items[0],
                    'label': int(items[1]),
                    'prompt': prompts['prompt'],
                    'mask_id': prompts['mask_id']
                }
        return Data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

In [ ]:
# testing
from google.colab import drive
drive.mount('/content/gdrive')
import os
os.chdir('/content/gdrive/MyDrive/sentiment-analysis')

train_data = upload_dataset('chnsenticorp/train/part.0')
valid_data = upload_dataset('chnsenticorp/dev/part.0')
test_data = upload_dataset('chnsenticorp/test/part.0')
print(f'train set size: {len(train_data)}')
print(f'valid set size: {len(valid_data)}')
print(f'test set size: {len(test_data)}')
print(next(iter(train_data)))

Mounted at /content/gdrive
train set size: 9600
valid set size: 1200
test set size: 1200
{'comment': '选择珠江花园的原因就是方便，有电动扶梯直接到达海边，周围餐馆、食廊、商场、超市、摊位一应俱全。酒店装修一般，但还算整洁。 泳池在大堂的屋顶，因此很小，不过女儿倒是喜欢。 包的早餐是西式的，还算丰富。 服务吗，一般', 'label': 1, 'prompt': '总体上来说很[MASK]: 选择珠江花园的原因就是方便，有电动扶梯直接到达海边，周围餐馆、食廊、商场、超市、摊位一应俱全。酒店装修一般，但还算整洁。 泳池在大堂的屋顶，因此很小，不过女儿倒是喜欢。 包的早餐是西式的，还算丰富。 服务吗，一般', 'mask_id': 6}


## Data preprocessing

In [ ]:
import torch
import numpy as np
from torch.utils.data import DataLoader
from transformers import AutoTokenizer
checkpoint = 'bert-base-chinese'
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
# find the id of the label defined in tokenizer
label = map_label(tokenizer)
pos_id, neg_id = label['1']['id'], label['0']['id']
# Dataloader, in NLP we usually use collate_fn to do the padding to make sure samples have the same sequence length
def collate(batch_samples): # operate on each batch
    batch_sentence = []
    batch_label = []
    batch_mask_id = []
    max_length = 0
    for sample in batch_samples:
        batch_sentence.append(sample['prompt'])
        encoding = tokenizer(sample['prompt'], truncation=True)
        max_length = max(max_length, len(encoding.tokens())) # dynamic padding to the longest
        mask_idx = encoding.char_to_token(sample['mask_id']) # convert the character id of mask in prompts to token id in tokenized sentence
        batch_mask_id.append(mask_idx) # if pre-fix prompt, mask id will be the same
        batch_label.append(sample['label'])
    batch_inputs = tokenizer(batch_sentence, max_length=max_length, padding=True, truncation=True, return_tensors="pt")
    label_id = [neg_id, pos_id]
    return {
        'batch_inputs':batch_inputs,
        'batch_mask_id': batch_mask_id,
        'label_id': label_id,
        'labels': batch_label

    }


In [ ]:
# a testing for dataloader
# batch_data = next(iter(train_loader))
# print('batch_X shape:', {k: v.shape for k, v in batch_data['batch_inputs'].items()})
# print(batch_data['batch_inputs'])
# print(batch_data['batch_mask_id'])
# print(batch_data['label_id'])
# print(batch_data['labels'])

batch_X shape: {'input_ids': torch.Size([32, 194]), 'token_type_ids': torch.Size([32, 194]), 'attention_mask': torch.Size([32, 194])}
{'input_ids': tensor([[ 101, 2600,  860,  ...,    0,    0,    0],
        [ 101, 2600,  860,  ...,    0,    0,    0],
        [ 101, 2600,  860,  ...,    0,    0,    0],
        ...,
        [ 101, 2600,  860,  ...,    0,    0,    0],
        [ 101, 2600,  860,  ...,    0,    0,    0],
        [ 101, 2600,  860,  ...,    0,    0,    0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}
[7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7,

## Build the model

Look into the structure of the pre-trained model

In [ ]:
from transformers import AutoModelForMaskedLM, BertModel # the difference is that the former class contains a head layer for specific MLM task;
# BertModel is the base class, and we need to define the head layer e.g. the softmax

checkpoint = 'bert-base-chinese'
pre_trained_model = BertModel.from_pretrained(checkpoint)
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
vocab_size = len(tokenizer)
print(pre_trained_model)
print(vocab_size)

model.safetensors:   0%|          | 0.00/412M [00:00<?, ?B/s]

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(21128, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
  

Frozen all the parameters + Add the classiifcation layer (trainable)

In [ ]:
for param in pre_trained_model.parameters():
    param.requires_grad_(False)

In [ ]:
class Model(torch.nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.fc = torch.nn.Linear(768, vocab_size)

    def forward(self, batch_data):
        input_data = batch_data['batch_inputs']
        with torch.no_grad():
            out = pre_trained_model(**input_data)

        out = self.fc(out.last_hidden_state[:, batch_data['batch_mask_id'][0]]) # only works when the masking position is fixed (batch, vocab_size)
        out = out[:,batch_data['label_id']] # we only want to explore the prediction on the label_id
        return out

In [ ]:
from transformers import AdamW
from tqdm import tqdm
# Hyper-parameters
model = Model(vocab_size)
optimizer = AdamW(model.parameters(), lr=5e-4)
loss_fun = torch.nn.CrossEntropyLoss()
num_epoch = 1
batch_size = 32
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model = model.to(device)
# encapsulate into the dataloader as input
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True, collate_fn=collate)
valid_loader = DataLoader(valid_data, batch_size=batch_size, shuffle=True, collate_fn=collate)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=True, collate_fn=collate)
# def to_device(batch_data):
#     new_batch_data = {}
#     for k, v in batch_data.items():
#         if k == 'batch_inputs':
#             new_batch_data[k] = {
#                 k_: v_.to(device) for k_, v_ in v.items()
#             }
#         else:
#             new_batch_data[k] = torch.tensor(v).to(device)

#     return new_batch_data

# if torch.cuda.is_available():
#     device = torch.device("cuda")
#     model = model.to(device)
# else: 'cpu'


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:


for epoch in range(num_epoch):
    total_loss = 0
    i = 1
    for batch_data in tqdm(train_loader, desc=f"Epoch {epoch + 1}/{num_epoch}"):
        model.train() # training mode
        output = model(batch_data)
        loss = loss_fun(output, torch.tensor(batch_data['labels']))

        loss.backward()
        total_loss+=loss.item()

        optimizer.step()
        optimizer.zero_grad()# set gradient to 0 when batch is updated


        if i % 10 == 0: # record every 10 training batches
            output = output.argmax(dim=1)
            acc = (output == torch.tensor(batch_data['labels'])).sum().item()/ len(batch_data['labels'])
            print(f'Training stage:Batch {i}, Loss: {loss.item()}, Accuracy: {acc}')

        if i % 30 == 0: # validate every 30 batches
            model.eval()
            batch_val = next(iter(valid_loader))
            with torch.no_grad():
                output = model(batch_val)
            loss = loss_fun(output, torch.tensor(batch_val['labels']))
            output = output.argmax(dim=1)
            acc = (output == torch.tensor(batch_val['labels'])).sum().item()/ len(batch_val['labels'])
            print(f'Validation stage:Batch {i}, Loss: {loss.item()}, Accuracy: {acc}')

        i+=1

    average_loss = total_loss/len(train_loader)
    print(f"Training Epoch {epoch + 1}/{num_epoch}, Average Loss: {average_loss:.4f}")

torch.save(model.state_dict(), 'model_parameters.pth')
torch.save(model, 'model_file.pth')


In [ ]:
!/opt/bin/nvidia-smi

# Method 2: LoRA fine-tuning